In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

%matplotlib inline

In [2]:
# Read Training data
X = pd.read_csv('train.csv')
Y = X[['label']]
X = X.drop(["label"], axis=1)

# Read Test data
# X_test = pd.read_csv('test.csv')

In [3]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(Y.values)
OneHotY = enc.transform(Y).toarray()

In [4]:
def layer_sizes(X, Y):
    n_x = X.shape[0]
    n_h = 256
    n_y = Y.shape[0]
    return (n_x, n_h, n_y)

In [5]:
def initialize_parameters(n_x, n_h, n_y):
    np.random.seed(2)
    
    W1 = np.random.randn(n_h, n_x) * 0.01
    b1 = np.zeros((n_h, 1))
    W2 = np.random.randn(n_y, n_h) * 0.01
    b2 = np.zeros((n_y, 1))
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [6]:
def relu(x):
    return x * (x > 0)

In [7]:
def softmax(z):
    z -= np.max(z)
    sm = (np.exp(z) / np.sum(np.exp(z), axis=0))
    return sm

In [8]:
def sigmoid(z):
    return 1. / (1 + np.exp(-z))

In [9]:
def forward_propagation(X, parameters):

    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]

    Z1 = np.dot(W1, X) + b1
    A1 = sigmoid(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = softmax(Z2)
    
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    
    return A2, cache

In [10]:
def compute_cost(A2, Y, parameters):

    m = Y.shape[1]

    logprobs = np.multiply(np.log(A2), Y)
    cost = - (1/m) * np.sum(logprobs)
    
    cost = np.squeeze(cost)
    
    return cost

In [11]:
def backward_propagation(parameters, cache, X, Y):

    m = X.shape[1]
    
    W1 = parameters["W1"]
    W2 = parameters["W2"]

    A1 = cache["A1"]
    A2 = cache["A2"]
    
    Z1 = cache["Z1"]

    dZ2 = A2 - Y
    dW2 = (1. / m) * (np.dot(dZ2, A1.T))
    db2 = (1. / m) * (np.sum(dZ2, axis=1, keepdims=True))
    
    dA1 = np.dot(W2.T, dZ2)
    dZ1 = dA1 * sigmoid(Z1) * (1 - sigmoid(Z1))
    dW1 = (1. / m) * (np.dot(dZ1, X.T))
    db1 = (1. / m) * (np.sum(dZ1, axis=1, keepdims=True))
    
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    
    return grads

In [12]:
def update_parameters(parameters, grads, learning_rate = 0.1):

    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]

    dW1 = grads["dW1"]
    db1 = grads["db1"]
    dW2 = grads["dW2"]
    db2 = grads["db2"]

    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1
    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [13]:
def predict(parameters, X):

    A2, cache = forward_propagation(X, parameters)
    predictions = np.argmax(A2, axis=0)
    
    return predictions

In [14]:
def nn_model(X, Y, n_h, num_iterations = 10000, print_cost=False):

    np.random.seed(3)
    n_x = layer_sizes(X, Y)[0]
    n_y = layer_sizes(X, Y)[2]
    
    parameters = initialize_parameters(n_x, n_h, n_y)
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]

    for i in range(0, num_iterations):
         
        A2, cache = forward_propagation(X, parameters)

        cost = compute_cost(A2, Y, parameters)
 
        grads = backward_propagation(parameters, cache, X, Y)
 
        parameters = update_parameters(parameters, grads)
        
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))

    return parameters

In [15]:
parameters = nn_model(X.head(8000).T.values, OneHotY[0:8000].T, n_h = 256, num_iterations = 1000, print_cost=True)

Cost after iteration 0: -0.000000
Cost after iteration 100: -0.000000
Cost after iteration 200: -0.000000
Cost after iteration 300: -0.000000
Cost after iteration 400: -0.000000
Cost after iteration 500: -0.000000
Cost after iteration 600: -0.000000
Cost after iteration 700: -0.000000
Cost after iteration 800: -0.000000
Cost after iteration 900: -0.000000


In [16]:
import csv
import urllib2
from itertools import islice

X_test = []
Y_pred = []
with open('test.csv', 'rt') as f:
    reader = csv.reader(f, delimiter=',')
    next(reader)
    for i in range(28):
        X_test = [row for row in islice(reader, 1000)]
        Y_hat = predict(parameters, np.asarray(X_test, dtype='float64').T)
        Y_pred.append(Y_hat)

In [17]:
frame = pd.DataFrame({'Label': np.asarray(Y_pred).reshape(-1)})
frame.insert(0, 'ImageId', range(1, 1 + len(frame)))
frame.to_csv('output.csv', index=False)